In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [3]:
data = pd.read_csv('dataset/data.csv',delimiter='\t')

In [4]:
data.head()

,url,title,time_of_post,author,content,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,Amazon's Streaming Video Library Now a Little ...,2013-01-07 19:11:36 UTC,Lauren Indvik,Having trouble finding something to watch on A...,731.0,12.0,219.0,0.663594,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,AP's Twitter to Begin Displaying Sponsored Tweets,2013-01-07 19:08:52 UTC,Seth Fiegerman,The Associated Press is the latest news organi...,731.0,9.0,255.0,0.604743,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,Apple's App Store Passes 40 Billion Downloads,2013-01-07 14:07:21 UTC,Seth Fiegerman,It looks like 2012 was a pretty good year for ...,731.0,9.0,211.0,0.575130,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,This Astronaut Is Rooting for Notre Dame Tonight,2013-01-07 22:23:38 UTC,Tariq Mali,"When it comes to college football, NASA astron...",731.0,9.0,531.0,0.503788,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,New U-Verse Apps Simplify Sharing Photos and V...,2013-01-07 17:45:57 UTC,Emily Price,LAS VEGAS — Sharing photos and videos on your ...,731.0,13.0,1072.0,0.415646,1.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [5]:
author_dict = {}

for a in data['author']:
    if a not in author_dict:
        author_dict[a] = {'count': 0}
    else:
        author_dict[a]['count'] += 1

for shares, author in zip(data['shares'], data['author']):
    if 'shares' in author_dict[author]:
        author_dict[author]['shares'] += shares
    else:
        author_dict[author]['shares'] = 0
l1 = []
l2 = []

for key, item in author_dict.items():
    l1.append((key, item['shares']))
    l2.append((key, item['count']))
l1.sort(key=lambda x:x[1], reverse=True)
l2.sort(key=lambda x:x[1], reverse=True)

print('Authors with the most shares\n')
for t in l1[:10]:
    print(t[0], t[1])
print('\nAuthors with the most posts\n')
for t in l2[:10]:
    print(t[0], t[1])

Authors with the most shares

Stan Schroeder 5346956
Neha Prakash 5177886
Sam Laird 4557402
Brian Anthony Hernandez 4097241
Todd Wasserman 4063772
Seth Fiegerman 3933494
Samantha Murphy 3905488
Lorenzo Franceschi-Bicchierai 3766276
Laura Vitto 2759786
Pete Pachal 2468350

Authors with the most posts

Neha Prakash 1500
Sam Laird 1458
Stan Schroeder 1440
Todd Wasserman 1317
Seth Fiegerman 1278
Brian Anthony Hernandez 1240
Samantha Murphy 1101
Lorenzo Franceschi-Bicchierai 976
Laura Vitto 829
Pete Pachal 718


In [23]:
# create a simple word graph from content words
# step 1: remove unnecessary symbols from words, such as ', ?, . etc
# step 2: create a directed graph where each node is a word
# step 3: try to make sentences by randomly walking the graph 

content = pd.read_csv('dataset/clean_content.csv', delimiter='\t')
content.head()

,url,content
0,http://mashable.com/2013/01/07/amazon-instant-...,"having,trouble,finding,something,to,watch,on,a..."
1,http://mashable.com/2013/01/07/ap-samsung-spon...,"the,associated,press,is,the,latest,news,organi..."
2,http://mashable.com/2013/01/07/apple-40-billio...,"it,looks,like,was,a,pretty,good,year,for,apple..."
3,http://mashable.com/2013/01/07/astronaut-notre...,"when,it,comes,to,college,football,nasa,astrona..."
4,http://mashable.com/2013/01/07/att-u-verse-apps/,"las,vegas,sharing,photos,and,videos,on,your,tv..."


In [24]:
# create a word dict
word_dict = {}
contents = []
for c in content['content']:
    try:
        words = c.split(',')
        contents.append(words)
        for wrd in words:
            if wrd in word_dict:
                word_dict[wrd] += 1
            else:
                word_dict[wrd] = 0
    except:
        print(c)

nan
nan
nan
nan


In [32]:
import time
# find neighboring words and make a graph out of them
def findNeighbors(word, c):
    found = {}
    for i in range(1, len(c)-1):
        if c[i] == word:
            
            if not c[i-1] in found:
                found[c[i-1]] = True

            elif not c[i+1] in found:
                found[c[i+1]] = True
    return found.keys()

graph_dict = {}
amount = len(word_dict.items())

for key, item in word_dict.items():
    t1 = time.time()
    if key not in graph_dict:
        
        graph_dict[key] = {}
    
    for content in contents:
        neighbors = findNeighbors(key, content)
        
        for n in neighbors:
            
            if n in graph_dict[key]:
                graph_dict[key][n] += 1
            else:
                graph_dict[key][n] = 0
    t2 = time.time()
    print('time left {} hr'.format(((amount*(t2-t1))/60.0)/60.0))
    

time left 110.67019811477925 hr
time left 79.74669007440409 hr
time left 81.45953312118849 hr
time left 77.27136395619975 hr
time left 114.67885262774097 hr
time left 78.13937068276935 hr
time left 127.80860853208436 hr
time left 72.31334497405423 hr
time left 78.91631348146332 hr
time left 85.59658304380045 hr
time left 123.7513982051611 hr
time left 72.77120315419303 hr
time left 77.82513915684488 hr
time left 76.5071885242727 hr


KeyboardInterrupt: 

In [12]:
test = pd.read_csv('clean_content.csv', delimiter='\t')

In [14]:
d = {}
for c in test['content']:
    if isinstance(c, str):
        words = c.split(',')
        for wrd in words:
            if wrd in d:
                d[wrd] += 1
            else:
                d[wrd] = 1